In [66]:
import numpy as np
from scipy.sparse import csr_matrix

c = csr_matrix(np.array([[1.1,2.2,3],
                         [4,5.1,6],
                         [7.3,8,9]])) # construct an example matrix
c.todok().items()

dict_items([((0, 0), 1.1), ((1, 0), 4.0), ((2, 0), 7.3), ((0, 1), 2.2), ((1, 1), 5.1), ((2, 1), 8.0), ((0, 2), 3.0), ((1, 2), 6.0), ((2, 2), 9.0)])

In [76]:
_d =  [x[1]  for x in c.todok().items()]
_xy = [x[0][0]  for x in c.todok().items()], [x[0][1]  for x in c.todok().items()]
_d, _xy

([1.1, 4.0, 7.3, 2.2, 5.1, 8.0, 3.0, 6.0, 9.0],
 ([0, 1, 2, 0, 1, 2, 0, 1, 2], [0, 0, 0, 1, 1, 1, 2, 2, 2]))

In [74]:
csr_matrix((_d,_xy)).toarray()

array([[1.1, 2.2, 3. ],
       [4. , 5.1, 6. ],
       [7.3, 8. , 9. ]])

In [36]:
_x=[]
_y=[]
_d=[]
for i,ent in enumerate(c.todok().items()):
    print(ent)
    _x.append(ent[0][0])
    _y.append(ent[0][1])
    _d.append(ent[1])
    
abc = (_d,(_x,_y))
print('->',abc)

csr_matrix(abc).toarray()

((0, 0), 1)
((1, 0), 4)
((2, 0), 7)
((0, 1), 2)
((1, 1), 5)
((2, 1), 8)
((0, 2), 3)
((1, 2), 6)
((2, 2), 9)
-> ([1, 4, 7, 2, 5, 8, 3, 6, 9], ([0, 1, 2, 0, 1, 2, 0, 1, 2], [0, 0, 0, 1, 1, 1, 2, 2, 2]))


array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)

In [79]:
import codecs
compressor = 'zlib'
import io
import numpy as np
import sqlite3

def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    # zlib uses similar disk size that Matlab v5 .mat files
    # bz2 compress 4 times zlib, but storing process is 20 times slower.
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(codecs.encode(out.read(),compressor))  # zlib, bz2

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    out = io.BytesIO(codecs.decode(out.read(),compressor))
    return np.load(out)

In [82]:
convert_array(adapt_array(_xy))

array([[0, 1, 2, 0, 1, 2, 0, 1, 2],
       [0, 0, 0, 1, 1, 1, 2, 2, 2]])

# TF-IDF

In [114]:
import sqlite3
from tqdm import tqdm
from annoy import AnnoyIndex

t = AnnoyIndex(7000, 'angular')
t.load('swarog_tfidf.ann')


True

In [89]:
t2 = AnnoyIndex(768, 'angular')
t2.load('swarog.ann') # super fast, will just mmap the file

True

In [121]:
from tqdm import tqdm
conn = sqlite3.connect('../swarog.sqlite')
c = conn.cursor()

c.execute("SELECT rowid, body, label from raw limit 1 offset 5000")
print(c.fetchall())

conn.close()

[(5001, 'The disaster management act in India prohibits posting messages regarding COVID-19 and would attract punishment to those who donâ€™t obey.', 1)]


In [122]:
t.get_nns_by_item(5000, 10, search_k=-1, include_distances=True)

([5000, 1306, 5438, 5821, 1271, 4932, 2451, 4952, 2459, 6922],
 [0.0,
  1.1187093257904053,
  1.12673819065094,
  1.139880895614624,
  1.1470364332199097,
  1.1535050868988037,
  1.162827730178833,
  1.172426462173462,
  1.1852103471755981,
  1.188748836517334])

In [129]:
from tqdm import tqdm
conn = sqlite3.connect('../swarog.sqlite')
c = conn.cursor()

c.execute("SELECT rowid-1, body, label from raw limit 1 offset 6922 ")
print(c.fetchall())

conn.close()

[(6922, 'India spraying pesticides at night to prevent COVID-19.', 1)]


In [124]:
t2.get_nns_by_item(5000, 10, search_k=-1, include_distances=True)

([5000, 6574, 1705, 7286, 5014, 3205, 5875, 4535, 5830, 7779],
 [0.0,
  0.30667322874069214,
  0.3668844997882843,
  0.3827296197414398,
  0.3927028179168701,
  0.3958447575569153,
  0.41854533553123474,
  0.4232296347618103,
  0.4259747564792633,
  0.43050748109817505])

In [128]:
from tqdm import tqdm
conn = sqlite3.connect('../swarog.sqlite')
c = conn.cursor()

c.execute("SELECT rowid-1, body, label from raw limit 1 offset 7779")
print(c.fetchall())

conn.close()

[(7779, 'UNICEF advises a certain set of steps to follow for coronavirus.', 1)]
